In [1]:
!pip install emoji
!pip install tensorflow
!pip install transformers

     |████████████████████████████████| 175 kB 31.6 MB/s 
  Created wheel for emoji: filename=emoji-1.7.0-py3-none-any.whl size=171046 sha256=36add0223a8c827be38496524dcb3f00ef8b362643a27810f095acf12edea283
  Stored in directory: /root/.cache/pip/wheels/8a/4e/b6/57b01db010d17ef6ea9b40300af725ef3e210cb1acfb7ac8b6
Successfully built emoji
     |████████████████████████████████| 462 kB 33.0 MB/s 
     |████████████████████████████████| 4.0 MB 31.0 MB/s 
     |████████████████████████████████| 6.6 MB 69.2 MB/s 
     |████████████████████████████████| 895 kB 54.4 MB/s 
     |████████████████████████████████| 596 kB 77.0 MB/s 
     |████████████████████████████████| 77 kB 8.0 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [2]:
from transformers import AutoTokenizer, TFAutoModelForSequenceClassification
  
tokenizer = AutoTokenizer.from_pretrained("rabindralamsal/finetuned-bertweet-sentiment-analysis")

model = TFAutoModelForSequenceClassification.from_pretrained("rabindralamsal/finetuned-bertweet-sentiment-analysis")

Downloading:   0%|          | 0.00/323 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/824k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.03M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/17.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/150 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/929 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/515M [00:00<?, ?B/s]

All model checkpoint layers were used when initializing TFRobertaForSequenceClassification.

All the layers of TFRobertaForSequenceClassification were initialized from the model checkpoint at rabindralamsal/finetuned-bertweet-sentiment-analysis.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaForSequenceClassification for predictions without further training.


In [3]:
import tensorflow as tf
import numpy as np
import pandas as pd

In [25]:
# Code to read csv file into Colaboratory:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [5]:
link = 'https://drive.google.com/file/d/17-X8-Fg4B11bqxt_fZ5B-X2WzeuRky0f/view?usp=sharing' 

In [26]:
# to get the id part of the file
id = link.split("/")[-2]
 
downloaded = drive.CreateFile({'id':id})
downloaded.GetContentFile('Train_Feb.csv') 
 
df = pd.read_csv('Train_Feb.csv')
print(df)

                                                      text
0        @k_c12345 Brother this called link 👇\nhttps://...
1        Ukraine invasion: Who is President Volodymyr Z...
2        So it now looks like we're in a new phase of t...
3        @vanderbellen @karlnehammer TOP RUSSIAN AGGRES...
4        @ZelenskyyUa @EmmanuelMacron Ukraine won't fal...
...                                                    ...
1782546  @UNESCO Dear UNESCO. I'm a peaceful citizen of...
1782547  This is not Ukraine vs Russia war. This is a c...
1782548  Russia 🇷🇺 is warn... Don't provoke the 4nation...
1782549  Africa Report: How Africa factors in #RussiaUk...
1782550  @BanteSaket @LukeMackay13 @Reporter_indian I l...

[1782551 rows x 1 columns]


In [7]:
df['text'] = df['text'].astype('str') 

In [8]:
#df_sample = df.sample(n=50000, random_state=2)

In [9]:
#df_sample

In [10]:
docs = list(df.loc[:, "text"].values)

In [11]:
#get index for halving dataset at index: 178255
from math import floor
floor(len(docs) / 10)


178255

In [12]:
178255 * 4

713020

In [13]:
docs = docs[534765:713020]

In [14]:
from tqdm import tqdm
scores = []

for i in tqdm(range(len(docs))):
    input = tokenizer.encode(docs[i], return_tensors="tf")
    output = model.predict(input)[0]
    prediction = tf.nn.softmax(output, axis=1).numpy()
    sentiment = np.argmax(prediction)
    
    scores.append({"Label": sentiment,
                       "Negative": prediction[0,0],
                       "Neutral": prediction[0,1],
                       "Positive": prediction[0,2]
                  })

100%|██████████| 178255/178255 [3:19:19<00:00, 14.91it/s]


In [28]:
df = df[534765:713020]

In [33]:
sentiments_score = pd.DataFrame.from_dict(scores)
df = df.join(sentiments_score.set_index(df.index))

In [34]:
df

,text,Label,Negative,Neutral,Positive
534765,"""UN General Assembly votes to condemn Russia o...",1,0.382025,0.611055,0.006920
534766,@SarahSp58588539 I think they will be making a...,1,0.414590,0.574292,0.011118
534767,#PutinHitler I don’t understand why Belarus is...,0,0.974843,0.021451,0.003706
534768,@Sonny1366 @TheRickWilson You are parroting Pu...,0,0.929840,0.067592,0.002569
534769,of course the conflict hasnt nearly been as se...,0,0.897131,0.099993,0.002877
...,...,...,...,...,...
713015,@BOUSMAH79199901 @SecBlinken What are you talk...,0,0.966040,0.031167,0.002793
713016,@Ukraine @MFA_Ukraine @APUkraine @GeneralStaff...,0,0.530390,0.434946,0.034664
713017,New signs put up to greet Russia forces in Ukr...,2,0.000823,0.045444,0.953734
713018,For #peace in the #world\nIn #Ukraine in #Rus...,2,0.000682,0.045640,0.953678


In [35]:
from google.colab import drive
drive.mount('drive')

df.to_csv('finetuned_Bertweet_SA_Feb_Train_p4.csv')
!cp finetuned_Bertweet_SA_Feb_Train_p4.csv 'drive/My Drive/Colab Notebooks'

Mounted at drive
